In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def RW1D(N, x0):
    """Genera una marcha aleatoria de 1D a partir de una posición inicial x0."""
    trayectoria = np.zeros(N, dtype=np.int64)
    trayectoria[0] = x0
    for i in range(1, N):
        trayectoria[i] = trayectoria[i - 1] + np.random.choice([-1, 1], p=[0.5, 0.5])
    return trayectoria

def different_position(Position):
    """Comprueba si dos numeros en un arreglo son iguales."""
    for i in range(len(Position)):
        for j in range(i+1, len(Position)):
            if Position[i] == Position[j] and i != j:
                return False
    return True

def sample_non_intersecting_walks_N(N, x0, T, t):
    """Genera marchas aleatorias condicionadas a no intersectarse en un paso
    específico usando aceptación-rechazo con trayectorias partiendo de x0 hasta
    un tiempo N."""

    #Condiciones a pedir

    assert len(x0) == N #Si hay la misma cantidad de caminantes que condiciones iniciales
    assert t > T #Generamos trayectorias más grandes que el tiempo que buscamos que se intersecten

    #se crea una matriz que contenga a todas las trayectorias de los caminantes
    Walks = np.zeros((N,t), dtype=np.int64)

    while True:

        NoIntersection = True
        j = 0

        #Se crea un paseo aleatoria para cada caminante

        for i in range(N):
            Walks[i] = RW1D(t,x0[i])
        #Se ve si es un caso de no intersección entre los caminos

        while NoIntersection and j < T + 1:
            NoIntersection = different_position(Walks[:,j])
            j += 1

        #En el caso que no haya intersección se sale del ciclo y se retorna los caminos.
        if NoIntersection == True:
            break

    return Walks

def find_first_intersection_N(N, Walks, T, t):
    """Esta función calcula la primera intersección entre
    entre dos pares de caminantes."""

    for i in range(T+1, t):

        if not(different_position(Walks[:,i])):
            time_intersection = i
            for l in range(N):
                for m in range(l+1,N):
                    if Walks[l,time_intersection] == Walks[m,time_intersection]:
                        pos_intersection = Walks[l,time_intersection]
                        return time_intersection, pos_intersection

    return None, None


def plot_random_walks_N(N, homogeneous_Walks, Walks, T, t, pos, time_intersection, T_final, pos_intersection, title_prefix, diff_inter_time):
    """Genera y grafica las dos marchas aleatorias y devuelve el tramo hasta la diferencia."""

    # Crear una figura con dos subgráficas
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))

    # Gráfico de las marchas aleatorias
    time_steps = np.arange(pos + t)

    for i in range(N):
        axs[0].plot(time_steps,  np.concatenate((homogeneous_Walks[i,:pos],Walks[i])), linewidth=2)

    axs[0].set_xlabel('Tiempo', fontsize=14)
    axs[0].set_ylabel('Posición', fontsize=14)
    axs[0].set_title(f'{title_prefix} (Sin Intersección hasta el Paso {T})', fontsize=16)
    axs[0].axvline(x=T, color='red', linestyle='--', label=f'Paso {T}', linewidth=1.5)
    if time_intersection is not None:
      axs[0].axvline(x=time_intersection + pos, color = "purple", linestyle='--', label=f'Intersección: t={time_intersection + pos}', linewidth=1.5)
      axs[0].scatter(time_intersection + pos, pos_intersection, color = "purple")
    if pos != 0:
      axs[0].axvline(x=diff_inter_time, color = "yellow", linestyle='solid', label=f'Comienzo caminatas aleatoria miopes t={diff_inter_time}', linewidth=1.5)
      axs[0].axvline(x=diff_inter_time + T, color = "c", linestyle='solid', label=f'Final del condicionamiento : t={diff_inter_time + T}', linewidth=1.5)


    time_steps_diferencia = np.arange(T_final + 1)

    for i in range(N):
        axs[1].plot(time_steps_diferencia, np.concatenate((homogeneous_Walks[i][0:pos],Walks[i]))[0:T_final +1], linewidth=2)

    axs[1].set_xlabel('Tiempo', fontsize=14)
    axs[1].set_ylabel('Posición', fontsize=14)
    axs[1].set_title(f'{title_prefix} (Desde Origen hasta Diferencia)', fontsize=16)
    axs[1].axvline(x=0, color='red', linestyle=':', label='Inicio', linewidth=1.5)
    axs[1].axvline(x=T_final, color='green', linestyle=':', label=f'Fin del Tramo t = {T_final}', linewidth=1.5)

    # Ajustar los gráficos
    for ax in axs:
        ax.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout(rect=[0, 0.05, 1, 0.95])  # Dejar espacio para la leyenda

    # Leyenda general
    handles, labels = axs[0].get_legend_handles_labels()
    handles2, labels2 = axs[1].get_legend_handles_labels()
    fig.legend(handles + handles2, labels + labels2, fontsize=12, loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.1))

    plt.show()


def sample_non_intersecting_walks_N_homogeneous(N, x0, T, t):
    """Función que entrega N caminatas aleatorias que no
    se intersectan hasta tiempo T, comenzando cada trayectoria
    en x0. Esta cadena se simula hasta un tiempo t > T."""

    #Se crea una matriz que guardará las trayectorias homogeneas y
    #listas para guardar ciertos datos

    homogeneous_Walks = np.zeros((N,t), dtype=np.int64)
    intersection_times = []
    intersection_pos   = []
    diff_inter_time = 0
    x0s = np.array(x0)
    T_final = T
    iteration = 0

    time = 0

    while time <= t:

        Walks = sample_non_intersecting_walks_N(N, x0s, T, t)
        time_intersection, pos_intersection = find_first_intersection_N(N, Walks, T, t)

        if time_intersection is not None:

            delta_time = time_intersection - T
            T_final = time_intersection - T + time
            if delta_time + time >= t:
                delta_time = t - time

        else:
            T_final = t
            delta_time = t - time

        plot_random_walks_N(N, homogeneous_Walks, Walks, T, t, time, time_intersection, T_final, pos_intersection, f'Iteración {iteration + 1}', diff_inter_time)

        if time_intersection is not None:
          diff_inter_time = T_final

        if delta_time + time >= t:
            homogeneous_Walks[:,time:delta_time + time +1] = Walks[:,0:delta_time]
            return homogeneous_Walks

        homogeneous_Walks[:,time:delta_time + time +1] = Walks[:,0:delta_time+1]

        x0s = Walks[:,delta_time]
        time += delta_time
        iteration += 1

    return homogeneous_Walks


In [ ]:
def N_walks_homogeneous(N, x0, t, T):
      """Función que entrega N caminatas aleatorias que no
    se intersectan hasta tiempo T, comenzando cada trayectoria
    en x0. Esta cadena se simula hasta un tiempo t > T."""
    homogeneous_Walks = np.zeros((N,t+1), dtype=np.int64)

    time = 0
    x0s = x0
    contador = 1

    while time < t:

        Walks = sample_non_intersecting_walks_N(N, x0s, T, t+1)
        time_intersection, pos_intersection = find_first_intersection_N(N, Walks, T, t + 1)


        if time_intersection is not None:
            delta_time = time_intersection - T

        else:
            delta_time = t - time


        if delta_time + time > t:
            delta_time = t - time

        homogeneous_Walks[:, time:time + delta_time + 1] = Walks[:, :delta_time + 1]

        x0s = Walks[:,delta_time]

        time += delta_time
        contador +=1

    return homogeneous_Walks

In [ ]:
def espaciado(Walks):
    """Esta función calcula el espacio generado por los caminantes generados homogeneamente"""
    Walks_final = Walks[:,-1]
    espacios = []
    for i in range(len(Walks_final)-1):
        espacios.append(Walks_final[i+1]-Walks_final[i])
    return espacios

def espacios_fijos(N,Walks):
  """Esta función calcula los espacios generados por particulas uniformente entre el maximo y el minimo de los caminantes homogeneos"""
    m = Walks[0,-1]
    M = Walks[-1,-1]

    puntos_intermedios = np.random.randint(m , M, size = N-2)

    puntos = np.concatenate(([m], puntos_intermedios, [M]))
    puntos_ordenados = np.sort(puntos)

    espaciado = []

    for i in range(N-1):
        espaciado.append(puntos_ordenados[i+1]-puntos_ordenados[i])

    return espaciado

def ley_espaciados(N,T,t,x0,M):
  """Esta función duevuelve los espacios generados por la cadena homogenea y la generada de forma uniforme repetida M veces"""

    espaciados = []
    espaciados_ind = []
    for i in range(M):
        print(i)
        Walks = N_walks_homogeneous(N,x0,t,T)
        espaciados.extend(espaciado(Walks))
        espaciados_ind.extend(espacios_fijos(N,Walks))

    return espaciados, espaciados_ind

def plot_histogram(espaciados, espaciados_ind, bin, T):
  """Función que gráfica histogramas"""

    bins = np.histogram_bin_edges([espaciados,espaciados_ind], bins = bin)

    plt.figure(figsize=(15,10))
    plt.hist(espaciados, bins=bins, alpha=0.6, color='blue', edgecolor='black',
            label='Cadena Homogénea')
    plt.hist(espaciados_ind, bins=bins, alpha=0.6, color='orange', edgecolor='black',
            label='Cadena No Homogénea')

    # Personalización
    plt.title(f'Comparación de distribuciones de espacios con T = {T}', fontsize=27)
    plt.xlabel('Espacio entre trayectorias', fontsize=23)
    plt.ylabel('Frecuencia', fontsize=23)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.legend(fontsize=20, loc='upper right')  # Leyenda con fuente grande
    plt.grid(axis='y', linestyle='--', alpha=0.7)  # Líneas de referencia para facilitar lectura

    # Mostrar el gráfico
    plt.tight_layout()  # Ajusta para evitar solapamientos
    plt.show()